# Introduction to LTspice.jl

a julia interface to [LTspice<sup>TM</sup>](http://www.linear.com/designtools/software/#LTspice)

<img src="circuit1.jpg">

In [1]:
using LTspice   # load the module

In [2]:
# create an simulation object 
# with either LTspiceSimulation or LTspiceSimulationTempDir
circuitpath = "circuit1.asc"
executablepath = "C:/Program Files (x86)/LTC/LTspiceIV/scad3.exe"
circuit1 = LTspiceSimulationTempDir(circuitpath,executablepath)

LTspiceSimulation:
circuit path = C:\Users\Chris\AppData\Local\Temp\julF669.tmp\circuit1.asc

Parameters
rsource                   = 50.0
rload                     = 75.0

Measurements
vload                     = NaN
pload                     = NaN


Notes: 
1. Using LTspiceSimulationTempDir to create an LTspiceSimulation object copies circuit to a temporary directory.
2. Names are all lower case within julia.

In [3]:
# circuit1 is a Dict like object
inputresistor = circuit1["rsource"]  # read a paramater by name

50.0

In [4]:
circuit1["rsource"] = 40.0  # write parameter by name.

40.0

In [5]:
loadpower = circuit1["pload"] # measurements can also be read by name
# since a valid measurement was not available, this caused a simulation run

0.00567108

In [6]:
circuit1

LTspiceSimulation:
circuit path = C:\Users\Chris\AppData\Local\Temp\julF669.tmp\circuit1.asc

Parameters
rsource                   = 40.0
rload                     = 75.0

Measurements
vload                     = 0.652174
pload                     = 0.00567108


In [7]:
circuit1["pload"] = 1.0 # WRONG.  measurements cannot be writen.

LoadError: LoadError: measurements cannot be set.
while loading In[7], in expression starting on line 1

In [8]:
haskey(circuit1,"rload")  # can check for keys

true

In [9]:
keys(circuit1),values(circuit1) # arrays of keys and values
# unlike standard Dict, these will always be in order.

(AbstractString["rsource","rload","vload","pload"],[40.0,75.0,0.652174,0.00567108])

In [10]:
parameternames(circuit1)  # array of parameter names

2-element Array{AbstractString,1}:
 "rsource"
 "rload"  

In [11]:
parametervalues(circuit1) # array of parameter values

2-element Array{Float64,1}:
 40.0
 75.0

In [12]:
measurementnames(circuit1) # array of measurement names

2-element Array{AbstractString,1}:
 "vload"
 "pload"

In [13]:
measurementvalues(circuit1) # Array{Float64,4} of measurement values.

2x1x1x1 Array{Float64,4}:
[:, :, 1, 1] =
 0.652174  
 0.00567108

In [14]:
rsource = 1.0
rload = 2.0
(vload,pload) = circuit1(rsource,rload) # simulation can be called

2-element Array{Float64,1}:
 0.666667
 0.222222

In [15]:
# call synthax is convenient for optimization
using Optim  # assumes Optim is installed
# find rload for max power
rsource = 50.0
result = optimize(rload -> -circuit1(rsource,rload)[2],10.0,100.0)
circuit1["rload"]

49.9784178958797

<img src="circuit2.jpg">

In [16]:
# create an LTspiceSimulation object for a stepped simulation
circuitpath = "circuit2.asc"
executablepath = "C:/Program Files (x86)/LTC/LTspiceIV/scad3.exe"
circuit2 = LTspiceSimulationTempDir(circuitpath,executablepath)

LTspiceSimulation:
circuit path = C:\Users\Chris\AppData\Local\Temp\jul196.tmp\circuit2.asc

Parameters
rsource                   = 50.0

Measurements
vload                     = NaN
pload                     = NaN

Sweeps


In [17]:
# parameters are a Dict for stepped simualtion
inputresistance = circuit2["rsource"] 

50.0

In [18]:
loadvoltage = circuit2["vload"] # WRONG. measurements are not

LoadError: LoadError: MethodError: `getindex` has no method matching getindex(::LTspice.SteppedLog, ::ASCIIString)
Closest candidates are:
  getindex(::LTspice.LogParsed, !Matched::Integer, !Matched::Integer, !Matched::Integer, !Matched::Integer)
while loading In[18], in expression starting on line 1

In [19]:
measurementvalues(circuit2) # use measurementvalues instead
# indexes are [measurement, inner_step, middle_step, outer_step]
# LTspice supports a maximum of 3 .step directives
# getmeasurements always returns Array{Float64,4}

2x7x3x1 Array{Float64,4}:
[:, :, 1, 1] =
 0.285714    0.375      0.444444    0.5    0.545455    0.583333    0.615385  
 0.00408163  0.0046875  0.00493827  0.005  0.00495868  0.00486111  0.00473373

[:, :, 2, 1] =
 0.285714    0.375      0.444444    0.5    0.545455    0.583333    0.615385  
 0.00408163  0.0046875  0.00493827  0.005  0.00495868  0.00486111  0.00473373

[:, :, 3, 1] =
 0.285714    0.375      0.444444    0.5    0.545455    0.583333    0.615385  
 0.00408163  0.0046875  0.00493827  0.005  0.00495868  0.00486111  0.00473373

In [20]:
stepnames(circuit2) # array of step names

2-element Array{AbstractString,1}:
 "rload"
 "vin"  

In [21]:
stepvalues(circuit2) # Tuple of Arrays of step values

([20.0,30.0,40.0,50.0,60.0,70.0,80.0],[1.0,2.0,3.0],Float64[])

In [22]:
# use PerLineIterator to write circuit2 to csv with headers
io = open("test.csv",false,true,true,false,false)
pli = PerLineIterator(circuit2) # create iterator to flatten 4d array for .csv 
writecsv(io,header(pli)) # write the headers
writecsv(io,pli) # write the data
close(io)

In [23]:
# this is what it looks like
println(header(pli))
for line in pli
    println(line)
end

AbstractString["rload" "vin" "rsource" "vload" "pload"]
[20.0,1.0,50.0,0.285714,0.00408163]
[30.0,1.0,50.0,0.375,0.0046875]
[40.0,1.0,50.0,0.444444,0.00493827]
[50.0,1.0,50.0,0.5,0.005]
[60.0,1.0,50.0,0.545455,0.00495868]
[70.0,1.0,50.0,0.583333,0.00486111]
[80.0,1.0,50.0,0.615385,0.00473373]
[20.0,2.0,50.0,0.285714,0.00408163]
[30.0,2.0,50.0,0.375,0.0046875]
[40.0,2.0,50.0,0.444444,0.00493827]
[50.0,2.0,50.0,0.5,0.005]
[60.0,2.0,50.0,0.545455,0.00495868]
[70.0,2.0,50.0,0.583333,0.00486111]
[80.0,2.0,50.0,0.615385,0.00473373]
[20.0,3.0,50.0,0.285714,0.00408163]
[30.0,3.0,50.0,0.375,0.0046875]
[40.0,3.0,50.0,0.444444,0.00493827]
[50.0,3.0,50.0,0.5,0.005]
[60.0,3.0,50.0,0.545455,0.00495868]
[70.0,3.0,50.0,0.583333,0.00486111]
[80.0,3.0,50.0,0.615385,0.00473373]
